**lab 2:**

1. Given the training set and the test set files from the Program 1 assignment, extract features from both using the standard tools and the mannual function (see above).

2. Make sure that both document-feature matrices need to be in the same Euclidean space, i.e., the $i$th dimension refers to the same token in both the training and test matrices. Think about different ways you could achieve this with the manual processing function.

3. Time processing the datasets using both the standard tools and the mannual method and report the times.

In [32]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from scipy.sparse import csr_matrix
import string
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
import math
from collections import defaultdict
import time
#import necessary modules

print(".")

.


In [45]:
#read lines from 'train.dat' into 'data'
with open("train.dat", "r") as fh:
    data = fh.readlines()

#preprocessing train.dat: remove numbers from beginning of lines
for i in range(len(data)):
    data[i] = data[i][2:]

#read lines from 'test.dat', append to 'data'
with open("test.dat", "r") as fh:
    data += fh.readlines()

#call count vectorizer to put data into matrix
#note time it takes for standard tools
start = time.time()
cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
count_occurrences = cv.fit_transform(data)
end = time.time()

count_vect_df = pd.DataFrame(data = count_occurrences.toarray(), columns= cv.get_feature_names_out())

print("with standard functions done in", end-start, "seconds")

with standard functions done in 3.548931837081909 seconds


In [35]:
count_vect_df[0:102081] #print rows of train.dat

,0,00,000,000016447,00010563n,0002,0004287,0005173miami050,000660,0009875buffalo040,...,zwick,zwiki,zy,zydrunas,zygmunt,zyman,zyprexa,zz,zzz,zzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102076,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102077,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102079,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
count_vect_df[102081:] #print rows of test.dat

,0,00,000,000016447,00010563n,0002,0004287,0005173miami050,000660,0009875buffalo040,...,zwick,zwiki,zy,zydrunas,zygmunt,zyman,zyprexa,zz,zzz,zzzzzz
102081,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102082,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102084,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#manually (from scratch)

def cmer(name, c=3):
    r""" Given a string and parameter c, return the vector of k-mers associated with the words
    """
    name = name.lower()
    if len(name) < c:
        return [name]
    v = []
    for i in range(len(name)-c+1):
        v.append(name[i:(i+c)])
    return v

def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
        
def textToMatrix(names, c):
    docs = [cmer(n, c) for n in names]
    return build_matrix(docs)

In [54]:
#measure time it takes to put data into matrix
start = time.time()
csr_info(textToMatrix(data, 1))
end = time.time()

print("manual method completed in", end-start, "seconds")



#make table to test
cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
count_occurrences = cv.fit_transform(data)
count_vect_df = pd.DataFrame(data = count_occurrences.toarray(), columns= cv.get_feature_names_out())

 [nrows 127600, ncols 57, nnz 3842355]
manual method completed in 7.759436845779419 seconds


In [38]:
count_vect_df[0:102081] #print rows from train.dat

,0,00,000,000016447,00010563n,0002,0004287,0005173miami050,000660,0009875buffalo040,...,zwick,zwiki,zy,zydrunas,zygmunt,zyman,zyprexa,zz,zzz,zzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102076,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102077,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102079,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
count_vect_df[102081:] #print rows from test.dat

,0,00,000,000016447,00010563n,0002,0004287,0005173miami050,000660,0009875buffalo040,...,zwick,zwiki,zy,zydrunas,zygmunt,zyman,zyprexa,zz,zzz,zzzzzz
102081,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102082,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102084,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
